In [1]:
import tensorflow as tf

/Users/admin/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
import numpy as np

In [3]:
char_arr = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n',
           'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']

In [4]:
num_dic = {n:i for i, n in enumerate(char_arr)}
dic_len = len(num_dic)

In [8]:
print(num_dic)

{'a': 0, 'b': 1, 'c': 2, 'd': 3, 'e': 4, 'f': 5, 'g': 6, 'h': 7, 'i': 8, 'j': 9, 'k': 10, 'l': 11, 'm': 12, 'n': 13, 'o': 14, 'p': 15, 'q': 16, 'r': 17, 's': 18, 't': 19, 'u': 20, 'v': 21, 'w': 22, 'x': 23, 'y': 24, 'z': 25}


In [6]:
seq_data = ['word', 'wood', 'deep', 'dive', 'cold', 'cool', 'load', 'love', 'kiss', 'kind']

In [9]:
def make_batch(seq_data):
    input_batch = []
    target_batch =[]
    
    for seq in seq_data:
        input = [num_dic[n] for n in seq[:-1]]
        target = num_dic[seq[-1]]
        input_batch.append(np.eye(dic_len)[input])
        target_batch.append(target)
        
    return input_batch, target_batch

In [10]:
learning_rate = 0.01
n_hidden = 128
total_epoch = 30
n_step = 3
n_input = n_class = dic_len

In [11]:
X = tf.placeholder(tf.float32, [None, n_step, n_input])
Y = tf.placeholder(tf.int32, [None])
W = tf.Variable(tf.random_normal([n_hidden, n_class]))
b = tf.Variable(tf.random_normal([n_class]))

In [12]:
cell1 = tf.nn.rnn_cell.BasicLSTMCell(n_hidden)
cell1 = tf.nn.rnn_cell.DropoutWrapper(cell1, output_keep_prob = 0.5)
cell2 = tf.nn.rnn_cell.BasicLSTMCell(n_hidden)

In [13]:
multi_cell = tf.nn.rnn_cell.MultiRNNCell([cell1, cell2])

In [14]:
outputs, states = tf.nn.dynamic_rnn(multi_cell, X, dtype = tf.float32)

In [15]:
outputs = tf.transpose(outputs, [1, 0, 2])
outputs = outputs[-1]
model = tf.matmul(outputs, W) + b

In [16]:
cost = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits=model, labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)

In [17]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

In [18]:
input_batch, target_batch = make_batch(seq_data)

In [19]:
for epoch in range(total_epoch):
    _, loss = sess.run([optimizer, cost], feed_dict = {X:input_batch, Y:target_batch})
    
    print('epoch:', '%04d' %(epoch + 1), 'cost=,' '{:.6f}'.format(loss))
    
print('complete')

epoch: 0001 cost=,4.170629
epoch: 0002 cost=,3.173904
epoch: 0003 cost=,1.931568
epoch: 0004 cost=,1.767231
epoch: 0005 cost=,0.697539
epoch: 0006 cost=,1.309958
epoch: 0007 cost=,0.700162
epoch: 0008 cost=,0.448745
epoch: 0009 cost=,0.535408
epoch: 0010 cost=,0.310634
epoch: 0011 cost=,0.304930
epoch: 0012 cost=,0.233149
epoch: 0013 cost=,0.395698
epoch: 0014 cost=,0.291108
epoch: 0015 cost=,0.098101
epoch: 0016 cost=,0.124268
epoch: 0017 cost=,0.230856
epoch: 0018 cost=,0.107235
epoch: 0019 cost=,0.104453
epoch: 0020 cost=,0.077865
epoch: 0021 cost=,0.227603
epoch: 0022 cost=,0.059893
epoch: 0023 cost=,0.028836
epoch: 0024 cost=,0.059769
epoch: 0025 cost=,0.062397
epoch: 0026 cost=,0.032837
epoch: 0027 cost=,0.019745
epoch: 0028 cost=,0.019049
epoch: 0029 cost=,0.033056
epoch: 0030 cost=,0.035872
complete


In [20]:
prediction = tf.cast(tf.argmax(model, 1), tf.int32)
prediction_check = tf.equal(prediction, Y)
accuracy = tf.reduce_mean(tf.cast(prediction_check, tf.float32))

In [21]:
input_batch, target_batch = make_batch(seq_data)

In [22]:
predict, accuracy_val = sess.run([prediction, accuracy], feed_dict = {X:input_batch, Y:target_batch})

In [23]:
predict_words = []
for idx, val in enumerate(seq_data):
    last_char = char_arr[predict[idx]]
    predict_words.append(val[:3] + last_char)
    
print('\n====result====')
print('input:',[w[:3] + '' for w in seq_data])
print('pre:', predict_words)
print('accu:', accuracy_val)


====result====
input: ['wor', 'woo', 'dee', 'div', 'col', 'coo', 'loa', 'lov', 'kis', 'kin']
pre: ['word', 'wood', 'deep', 'dive', 'cold', 'cool', 'load', 'love', 'kiss', 'kind']
accu: 1.0
